In [60]:
import os
import csv
import json
from io import StringIO
from zipfile import ZipFile
from os.path import join
from collections import defaultdict, Counter
from glob import glob
from heapq import nsmallest, nlargest

In [50]:
REPORTS = '../data/facebook/reports'
NULL_DISCLAIMER = 'These ads ran without a disclaimer'

COUNTRIES = [
    'AT',
    'BE',
    'BG',
    'CY',
    'CZ',
    'DE',
    'DK',
    'EE',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'HR',
    'HU',
    'IE',
    'IT',
    'LT',
    'LU',
    'LV',
    'MT',
    'NL',
    'PL',
    'PT',
    'RO',
    'SE',
    'SI',
    'SK',
    'US'
]

CURRENCY_RATES = {
    'EUR': 1,
    'USD': 0.88,
    'CZK': 0.039,
    'DKK': 0.134,
    'GBP': 1.12,
    'HUF': 0.0031,
    'PLN': 0.235,
    'RON': 0.21,
    'SEK': 0.095
}

In [54]:
# Finding unique advertisers and the country they operate in
ADVERTISERS = {}

for country in COUNTRIES:
    folder = join(REPORTS, country)
    
    last_date = sorted(d for d, _, _, in os.walk(folder))[-1].split('/')[-1]
    zip_path = glob(join(REPORTS, country, last_date, '*_lifelong.zip'))[0]

    with ZipFile(zip_path) as zipfile:
        csv_entry = next(e for e in zipfile.infolist() if e.filename.endswith('.csv'))
        
        with zipfile.open(csv_entry.filename) as csv_f:
            csv_io = StringIO(csv_f.read().decode('utf-8-sig'))
            reader = csv.DictReader(csv_io)
            currency_field = next(f for f in reader.fieldnames if f.startswith('Amount Spent'))
            currency = currency_field.split('(')[-1].split(')')[0]
            
            for line in reader:
                disclaimer = line['Disclaimer']
                page_name = line['Page Name']
                page_id = line.get('Page ID')
                spent = int(line[currency_field].replace('≤', ''))
                ads = int(line['Number of Ads in Library'].replace('≤', ''))
                
                has_disclaimer = disclaimer != NULL_DISCLAIMER
                advertiser = disclaimer if has_disclaimer else page_name
                
                record = ADVERTISERS.get(advertiser)

                if record is None:
                    record = {
                        'disclaimer': disclaimer,
                        'page_name': page_name,
                        'page_id': page_id,
                        'advertiser': advertiser,
                        'advertiser_kind': 'disclaimer' if has_disclaimer else 'page',
                        'countries': set(),
                        'spent': Counter(),
                        'spent_euro': Counter(),
                        'ads': Counter()
                    }
                    ADVERTISERS[advertiser] = record
                    
                record['countries'].add(country)
                record['spent'][country] += spent
                record['ads'][country] += ads
                record['spent_euro'][country] += spent * CURRENCY_RATES[currency]

In [59]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        
        return json.JSONEncoder.default(self, obj)

with open('../international_facebook_advertising.json', 'w') as jsonf:
    json.dump(ADVERTISERS, jsonf, ensure_ascii=False, cls=CustomJSONEncoder)

In [76]:
def sum_spent(a):
    return sum(a['spent_euro'].values())

top = nlargest(25, (a for a in ADVERTISERS.values() if len(a['countries']) > 1), key=sum_spent)

for a in top:
    print(a['advertiser'], sum_spent(a))
    for c in a['countries']:
        print('  %s' % c)

Facebook 10964686.29
  DK
  US
  NL
  ES
  IT
  GB
  FR
  PL
  DE
  SE
  BE
Care2 4573409.52
  US
  GB
International Rescue Committee 3448097.32
  AT
  DK
  US
  ES
  IT
  FR
  BE
European Parliament 3169695.9619
  SI
  DK
  FR
  CY
  DE
  SK
  HR
  LT
  BE
  RO
  EE
  LV
  IT
  GB
  MT
  HU
  BG
  PT
  ES
  FI
  PL
  SE
  AT
  GR
  IE
  LU
  NL
  CZ
Ben & Jerry's 1032531.2799999999
  NL
  ES
  US
  GB
USA for UNHCR 975856.48
  GR
  BG
  EE
  US
  NL
  ES
  IT
  BE
Friends of the Earth 767723.36
  US
  GB
TOMS 520587.12
  FR
  NL
  US
Wholesome Culture 411828.41
  SI
  DK
  US
  FR
  DE
  SK
  HR
  LT
  BE
  RO
  EE
  LV
  IT
  MT
  HU
  BG
  PT
  ES
  FI
  PL
  SE
  AT
  GR
  IE
  LU
  NL
  CZ
Patagonia 395254.32
  IE
  US
the Liberal Democrats 347292.93
  GR
  IE
  DK
  HU
  BG
  PT
  RO
  NL
  ES
  IT
  GB
  FR
  PL
  DE
  SK
  MT
  LT
  BE
The Labour Party 307466.07999999996
  IE
  GB
Kialo 269836.01
  SI
  DK
  US
  FR
  CY
  DE
  SK
  HR
  LT
  BE
  RO
  EE
  LV
  IT
  GB
  MT
  